# Datatrack.py Tutorial
Datatracks are the bread and butter of a great deal of research computational biology. After much of the preprocessing of experimental sequencing data we are usually left with a file which in some way details how some measurement of interest varies across the genome. The simplest way of thinking about a datatrack is just as a function which can take real values. If we let $\mathbf{C}$ be the set of chromosomes then we might define a datatrack as a function:


\begin{equation}
    f: \mathbb{N}_{+}\times\mathbf{C} \to \mathbb{R}
\end{equation}

Where we input a positive integer and chromosome and we get a real value. This value could, for example, tell us about the strength of binding of a given protein to a given basepair. Or it could be the number of reads which we observe at that basepair.

### The DataTrack Class
Simple and quick manipulation of the functions which we come across for various ChIPseq datasets (or, for example, DNA methylation dataset datasets) vastly increases the speed at which different hypotheses can be tested. Being able to keep track of variables also greatly reduces mental load! The Datatrack class is a quick way to load up data from standard data sources and interrogate them to find out properties of the underlying functions which they represent. 

### dtrack_utils
The Datatrack class makes heavy use of a cython module called GrapHiC.utils.dtrack_utils. Mastering this module will allow users to implement much more complicated things than in this notebook. However, the datatrack class is useful as-is for a lot of analysis. It's been implemented in cython so that many of the most common tasks such as overlapping two datatracks or collecting statistics on some group of regions can be done faster than would be possible in python.


## Importing Modules

In [1]:
#dtrack_utils Cython Module
#from GrapHiC.utils import dtrack_utils as dtu
#Numpy is always useful for numerical manipulation
import numpy as np
import itertools
import pandas as pd
#Import our datatrack classes!
from GrapHiC.utils.Datatrack import DataTrack_rvp as dtrvp
from GrapHiC.utils.Datatrack import DataTrack_bigwig as dtbw

from GrapHiC.Datatrack_creation import evaluate_tracks_over_cooler_bins, evaluate_tracks_over_bed_dataframe

# Basic Datatrack creation over cooler bins

One of the basic things to do with datatracks is to collect statistics for those tracks over hi-C data for each bin. We can do this using the evaluate_tracks_over_cooler_bins function:

In [9]:
contact_path = '../tutorial_data/WT_contacts.cool'
nanog_peaks = "../tutorial_data/GSEXXXXXHendrich20161026_Nanog_ESC_peaks.bed"
nanog_bigwig = "../tutorial_data/GSE71932_Nanog_mESCs_treat_pileup_filter.bw"

In [10]:
df = evaluate_tracks_over_cooler_bins(contact_path,
                             paths = [nanog_peaks,nanog_bigwig],
                             names = ['nanog_bigwig','nanog_peaks'],
                             stats_types = ['mean','max','std'],
                             allowed_chroms = ['chr1','chr2','chrX'],
                                      value_col = 4
                            )
df

nanog_bigwig ../tutorial_data/GSEXXXXXHendrich20161026_Nanog_ESC_peaks.bed
	mean
	max
	std
nanog_peaks ../tutorial_data/GSE71932_Nanog_mESCs_treat_pileup_filter.bw
	mean
	max
	std


,nanog_bigwig_mean,nanog_bigwig_max,nanog_bigwig_std,nanog_peaks_mean,nanog_peaks_max,nanog_peaks_std
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
26346,0.0,0.0,0.0,0.0,0.0,0.0
26347,0.0,0.0,0.0,0.10049,6.0,0.586188
26348,0.0,0.0,0.0,0.092287,6.0,0.501517
26349,0.0,0.0,0.0,0.147928,3.0,0.489231


## Evaluation over arbitrary regions
alternatively, we can evaluate datatracks over arbitrary regions specificed by a bed file. Let's take a look at some regions of active promoters:

In [11]:
df = pd.read_table("../tutorial_data/active_promoters.csv")

colnames, arr = evaluate_tracks_over_bed_dataframe(df,
                             paths = [nanog_peaks,nanog_bigwig],
                             names = ['nanog_bigwig','nanog_peaks'],
                             stats_types = ['mean','max','std'],
                             value_col = 4
                            )
pd.DataFrame(data = arr, columns = colnames)

nanog_bigwig ../tutorial_data/GSEXXXXXHendrich20161026_Nanog_ESC_peaks.bed
	mean
	max
	std
nanog_peaks ../tutorial_data/GSE71932_Nanog_mESCs_treat_pileup_filter.bw
	mean
	max
	std


,nanog_bigwig_mean,nanog_bigwig_max,nanog_bigwig_std,nanog_peaks_mean,nanog_peaks_max,nanog_peaks_std
0,0.00000,0.0,0.000000,7.225386,37.0,7.559524
1,0.00000,0.0,0.000000,4.174326,13.0,2.608505
2,0.00000,0.0,0.000000,3.006454,9.0,1.850368
3,0.00000,0.0,0.000000,3.074620,8.0,1.882457
4,7.74675,99.0,26.587894,4.073532,16.0,3.523583
...,...,...,...,...,...,...
10208,0.00000,0.0,0.000000,2.200084,6.0,1.574169
10209,0.00000,0.0,0.000000,1.917015,8.0,1.600306
10210,0.00000,0.0,0.000000,2.287324,18.0,2.841367
10211,0.00000,0.0,0.000000,1.609052,5.0,1.164586


# Region-Value Pair Datatracks
## BED format files
One of the most common formats for storing datatracks is in region-value-pair format. That is, on each chromosome we simple detail the regions where the function $f$ takes nonzero values and we specify the value that the function takes in that region. Below is an example of bed file created for a Nanog ChIPseq from naive embryonic stem cells:

In [12]:
!head ../tutorial_data/GSEXXXXXHendrich20161026_Nanog_ESC_peaks.bed

chr1	3062626	3063246	GSEXXXXXHendrich20161026_Nanog_ESC_peak_1	1091
chr1	3445421	3446366	GSEXXXXXHendrich20161026_Nanog_ESC_peak_2	658
chr1	3681365	3681811	GSEXXXXXHendrich20161026_Nanog_ESC_peak_3	48
chr1	3947036	3947306	GSEXXXXXHendrich20161026_Nanog_ESC_peak_4	55
chr1	4150783	4151277	GSEXXXXXHendrich20161026_Nanog_ESC_peak_5	1017
chr1	4447859	4448346	GSEXXXXXHendrich20161026_Nanog_ESC_peak_6	329
chr1	4611937	4612228	GSEXXXXXHendrich20161026_Nanog_ESC_peak_7	104
chr1	4660171	4660590	GSEXXXXXHendrich20161026_Nanog_ESC_peak_8	195
chr1	4802495	4802928	GSEXXXXXHendrich20161026_Nanog_ESC_peak_9	424
chr1	4819456	4819933	GSEXXXXXHendrich20161026_Nanog_ESC_peak_10	346


We can see that there is chromosome information in collumn zero, then a region start collumn, a region end collumn, some unique IDs for each peak and then a value collumn detailing the size of the function at each peak. Lets go ahead and load this information into a DataTrack_rvp object (the rvp stands for region-value-pairs) using the .from_bed() method:

In [13]:
#In order to initialise our Datatrack all we need to do is specify a name. So lets create datatrack object
#with name nanog and save it in the variable nanog
nanog = dtrvp("Nanog")

#Lets now construct the datatrack from the bed file
nanog.from_bed(nanog_peaks,
               chrom_col = 0, #What collumn is our chromosome information in?
               region_cols = (1,2), #What collumns are our region start and region end?
               value_col = 4, #What collumns are our values in?
               ID_col = 3) #What collumns are our region IDs in?)

Note that we could also ignore the ID and value information and simply put all values as a constant:

In [14]:
nanog = dtrvp("Nanog").from_bed(nanog_peaks,
                                chrom_col = 0, #What collumn is our chromosome information in?
                                region_cols = (1,2), #What collumns are our region start and region end?
                                value_col = None, #What collumns are our values in?
                                value_fill = 1,
                                ID_col = None)

Now we can take a look at the data which is in our datatrack object:

In [15]:
#Lets take a look at our regions first
nanog.regions['chr1']

array([[  3062626,   3063246],
       [  3445421,   3446366],
       [  3681365,   3681811],
       ...,
       [194354695, 194354978],
       [194472371, 194472801],
       [194890314, 194890780]], dtype=int32)

In [16]:
#Now our values in chromosome 2
nanog.values['chr2']

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [17]:
#We can also look at the name and type of our datatrack if we forget
print("'nanog' is a datatrack of type: {}".format(nanog.dtrack_type))
print("It has name: {}".format(nanog.name))

'nanog' is a datatrack of type: region-value-pairs
It has name: Nanog


## Interrogating the datatrack function
Lets now look at functions for investigating region-value-pair functions. 

### Chromosome Limits
One of the first things we might be interested in is what the lengths of our chromosomes actually are! The Datatrack class doesn't assume that the chromosomes have any particular length and leaves these as functions which are free to be input by the user. However, we can estimate the chromosome limits from the maximum and minimum values of the regions we observe on each chromosome:

In [18]:
nanog.chrlims_from_regions()
nanog.chr_lims

{'chr1': [3062626, 194890780],
 'chr10': [3302560, 130420449],
 'chr11': [3145328, 121964594],
 'chr12': [3124034, 119777974],
 'chr13': [3712816, 119334473],
 'chr14': [7809188, 124446400],
 'chr15': [3748080, 103383326],
 'chr16': [3531957, 97936334],
 'chr17': [3505534, 94866952],
 'chr18': [3006850, 90581368],
 'chr19': [3532657, 61321440],
 'chr2': [3100032, 181773870],
 'chr3': [3370820, 159615381],
 'chr4': [3368156, 156194661],
 'chr5': [3293540, 151645339],
 'chr6': [4118617, 149357512],
 'chr7': [3507008, 145139802],
 'chr8': [3100430, 128538866],
 'chr9': [3376306, 123731738],
 'chrX': [6279324, 169906459],
 'chrY': [4829304, 4829578]}

### Datatrack function evaluation

Now that we have estimated our chromosome limits let's look at interrogating our function. Looking at the function for all values in a given region is simple and just uses the .func method. We just provide a midpoint and a windowsize (or buffer) around that midpoint where we want to evaluate the function:

In [19]:
#Here is the start of a peak in our Nanog datatrack on chromosome 1
nanog.func('1',[3062586], buffer = 100)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]])

Note that we can provide multiple midpoints and we'll get multiple rows in our output array:

In [20]:
#Here is the start of a peak in our Nanog datatrack on chromosome 1
nanog.func('1',[3062686,194354695], buffer = 10)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

This comes in use when we might want to evaluate a function across an entire set of genomic regions. For example active promoters:

In [21]:
x = pd.read_table('tutorial_data/active_promoters.csv')
x = {k1: np.concatenate([item[None,1:3] for item in list(g1)],
                                       axis = 0).astype('int32') for k1,g1 in itertools.groupby(sorted(x.values,
                                                                                                       key = lambda x:x[0]),
                                                                                                lambda x: x[0])}
x['chr1']

FileNotFoundError: [Errno 2] No such file or directory: 'tutorial_data/active_promoters.csv'

These are the active promoter regions on chromosome 1 in naive ES cells. Lets look at nanog signal within +/-1kb of the active promoter centres on chromosome 1:

In [ ]:
buffer = 1000
promoter_nanog_signal = np.zeros(2*buffer)
numproms = 0
for chrom in nanog.chromosomes:
    try:
        mids = np.mean(x[chrom], axis = 1).astype('int')
    except:
        print("couldn't find active promoter info for chromosome {}".format(chrom))
        continue
        
    promoter_nanog_signal += np.sum(nanog.func(chrom, mids, buffer = buffer), axis = 0)
    numproms += len(mids)

promoter_nanog_signal /= numproms

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (12,10))
plot = plt.plot(np.arange(len(promoter_nanog_signal)), promoter_nanog_signal)
xticks = plt.xticks([0,500,1000,1500,2000], [-1000, -500, 0, 500,1000])
plt.tick_params(labelsize = 15)
plt.ylabel("Average Signal", size = 20)
plt.xlabel("Distance from active promoter centre", size = 20)
plt.title("Active promoter nanog signal\nNaive mESCs", size = 20)

We might also just be interested in looking at coverage over certain regions. We can therefore replace the datatrack values with constants if we want to when evaluating the function:

In [ ]:
buffer = 1000
promoter_nanog_signal = np.zeros(2*buffer)
numproms = 0
for chrom in nanog.chromosomes:
    try:
        mids = np.mean(x[chrom], axis = 1).astype('int')
    except:
        print("couldn't find active promoter info for chromosome {}".format(chrom))
        continue
        
    promoter_nanog_signal += np.sum(nanog.func(chrom, mids, buffer = buffer, use_constant_val = 1), axis = 0)
    numproms += len(mids)

promoter_nanog_signal /= numproms
fig = plt.figure(figsize = (12,10))
plot = plt.plot(np.arange(len(promoter_nanog_signal)), promoter_nanog_signal)
xticks = plt.xticks([0,500,1000,1500,2000], [-1000, -500, 0, 500,1000])
plt.tick_params(labelsize = 15)
plt.ylabel("Average Coverage", size = 20)
plt.xlabel("Distance from active promoter centre", size = 20)
plt.title("Chromosome 1 active promoter nanog signal\nNaive mESCs", size = 20)

### Raw data within an interval 
We might instead want to get the actual raw region-value-pair information which overlaps with a region:

In [ ]:
nanog.data_in_interval('chr1', [3062586, 3062686])

### Binning Data within an interval
One more thing we might like to do is bin all of our information within a given region. For example, we might want to bin the entirety of chromosome 19 at a bin-size of 1000000 basepairs per bin and we're interested in the average Nanog signal per basepair in each bin:

#### mean per-basepair signal

In [ ]:
nanog.bin_single_interval('chr19',
                          int(1e6),
                          interval = nanog.chr_lims['chr19'],
                          stats_type = 'mean')[1]

We could also look at the coverage (in basepairs) within each bin:

#### Coverage

In [ ]:
nanog.bin_single_interval('chr19',
                          int(1e6),
                          interval = nanog.chr_lims['chr19'],
                          stats_type = 'coverage')[1]

We may also be interested in the mean signal within basepairs where we observe any signal at all:

#### Mean per-region signal

In [ ]:
nanog.bin_single_interval('chr19',
                          int(1e6),
                          interval = nanog.chr_lims['chr19'],
                          stats_type = 'per_region_mean')[1]

Finally we might be interested in the sum of all basepair signals in our data over each bin
#### Sum of all signals

In [ ]:
nanog.bin_single_interval('chr19',
                          int(1e6),
                          interval = nanog.chr_lims['chr19'],
                          stats_type = 'sum')[1]

## Statistics over an interval
We can also just input random intervals and get statistics across these intervals. Here we can use the stats function to pass an interval as well as a statistic of interest (stats_type) for regions within this interval. As with the func method we can also access multiple intervals at once:

### Coverage

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'coverage')

### Maximum value

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'max')

### Minimum value

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'min')

### Minimum value in regions with coverage

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'per_region_min')

### Standard deviation of values

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'std')

### Standard deviation of values in regions with coverage

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'per_region_std')

### Mean value

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'mean')

### Mean value in regions with coverage

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'per_region_mean')

### Sum of all values

In [ ]:
nanog.stats('chr1', [[0,1e7],[1e7,2e7]], stats_type= 'sum')

# BigWig files

BigWig data isn't actually included in the tutorial data since it is a

In [ ]:
nanog = dtbw("Nanog").from_bw(nanog_bigwig)

In [ ]:
nanog.chr_lims

In [ ]:
nanog.header

In [ ]:
#Here is the start of a peak in our Nanog datatrack on chromosome 1
nanog.func('chr1',[3062586], buffer = 100)

Lets reload our promoter info and see if we can form similar plots as above when we used the BED files:

In [ ]:
buffer = 1000
promoter_nanog_signal = np.zeros(100)
numproms = 0
for chrom in nanog.chromosomes:
    print(chrom)
    try:
        mids = np.mean(x[chrom], axis = 1).astype('int')
    except:
        print("couldn't find active promoter info for chromosome {}".format(chrom))
        continue
        
    promoter_nanog_signal += np.sum(nanog.func(chrom, mids, buffer = buffer, binSize = 10), axis = 0)
    numproms += len(mids)

promoter_nanog_signal /= numproms

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (12,10))
plot = plt.plot(np.arange(len(promoter_nanog_signal)), promoter_nanog_signal)
xticks = plt.xticks([0,50,100,150,200], [-1000, -500, 0, 500,1000])
plt.tick_params(labelsize = 15)
plt.ylabel("Average Signal", size = 20)
plt.xlabel("Distance from active promoter centre", size = 20)
plt.title("Active promoter nanog signal\nNaive mESCs", size = 20)